In [1]:
install.packages("tidyverse")
install.packages("broom")
install.packages("rsample")
install.packages("yardstick")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘warp’, ‘furrr’, ‘slider’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sparsevctrs’, ‘hardhat’




In [2]:
library(tidyverse)
library(broom)
library(rsample)
library(yardstick)

data <- read_csv("/content/data/akurba_with_referral.csv")

glimpse(data)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘yardstick’


The following object is masked from ‘package:readr’:

    spec


Rows: 243 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): Patient_ID, Sex, va_re, va_le, diagnosis, diagnosis_cat, age_group,...
dbl (5): Age, va_re_rank, va_le_rank, better_eye_rank, referral_needed

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `

Rows: 243
Columns: 13
$ Patient_ID      <chr> "P001", "P002", "P003", "P004", "P005", "P006", "P007"…
$ Age             <dbl> 45, 70, 54, 50, 72, 50, 53, 30, 47, 60, 37, 4, 4, 35, …
$ Sex             <chr> "Female", "Male", "Female", "Female", "Female", "Male"…
$ va_re           <chr> "6/36", "6/36", "6/24", "6/36", "6/36", "6/9", "HM", "…
$ va_le           <chr> "6/12", "6/60", "6/18", "6/36", "6/24", "6/9", "6/6", …
$ diagnosis       <chr> "Binasal Pterygium", "Le Senile Cataract", "Presbyopia…
$ diagnosis_cat   <chr> "Conjunctival/surface disorders", "Cataract-related", …
$ age_group       <chr> "Middle_aged", "Elderly", "Middle_aged", "Middle_aged"…
$ va_re_rank      <dbl> 6, 6, 5, 6, 6, 2, 8, 1, 1, 7, 1, NA, NA, 1, 1, 1, 1, 6…
$ va_le_rank      <dbl> 3, 7, 4, 6, 5, 2, 1, 1, 1, 5, 1, NA, NA, 2, 1, 1, 1, 6…
$ better_eye_rank <dbl> 3, 6, 4, 6, 5, 2, 1, 1, 1, 5, 1, NA, NA, 1, 1, 1, 1, 6…
$ better_eye_cat  <chr> "Moderate", "Severe", "Moderate", "Severe", "Severe", …
$ referral_needed 

In [5]:
data <- data %>%
  mutate(
    referral_needed = factor(referral_needed, levels = c(0, 1)),
    sex = factor(Sex),
    age_group = factor(age_group,
      levels = c("Child", "Adult", "Middle_aged", "Elderly")
    ),
    better_eye_cat = factor(
      better_eye_cat,
      levels = c("Normal_mild", "Moderate", "Severe", "Very_poor")
    ),
    diagnosis_cat = factor(diagnosis_cat)
  )

table(data$referral_needed)
table(data$referral_needed, data$age_group)
table(data$referral_needed, data$better_eye_cat)



  0   1 
119 124 

   
    Child Adult Middle_aged Elderly
  0    18    66          32       3
  1    15    21          56      32

   
    Normal_mild Moderate Severe Very_poor
  0         113        6      0         0
  1          24       32     32        18

In [7]:
set.seed(2025)

split <- initial_split(data, prop = 0.7, strata = referral_needed)
train_data <- training(split)
test_data  <- testing(split)

table(train_data$referral_needed)
table(test_data$referral_needed)


 0  1 
83 86 


 0  1 
36 38 

In [8]:
model <- glm(
  referral_needed ~ age_group + sex + better_eye_cat + diagnosis_cat,
  data = train_data,
  family = binomial
)

summary(model)


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”



Call:
glm(formula = referral_needed ~ age_group + sex + better_eye_cat + 
    diagnosis_cat, family = binomial, data = train_data)

Coefficients:
                                              Estimate Std. Error z value
(Intercept)                                  1.970e+01  1.003e+04   0.002
age_groupAdult                               1.378e+00  1.049e+04   0.000
age_groupMiddle_aged                         1.512e+00  2.190e+00   0.691
age_groupElderly                             8.013e-15  2.413e+00   0.000
sexMale                                      1.862e+00  1.763e+00   1.057
better_eye_catModerate                       2.331e+01  8.498e+03   0.003
better_eye_catSevere                         4.469e+01  1.365e+04   0.003
better_eye_catVery_poor                      2.755e+00  2.278e+04   0.000
diagnosis_catConjunctival/surface disorders -4.393e+01  1.225e+04  -0.004
diagnosis_catCorneal disorders               3.096e+00  5.391e+04   0.000
diagnosis_catGlaucoma-related          

In [9]:
model_tidy <- tidy(model, exponentiate = TRUE, conf.int = TRUE)
model_tidy

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),3.585495e+08,10031.813170,1.963511e-03,0.9984333,1.894499e-207,8.897336e+228
age_groupAdult,3.965000e+00,10493.850288,1.312679e-04,0.9998953,3.371705e-239,4.662694e+239
age_groupMiddle_aged,4.537656e+00,2.189975,6.906063e-01,0.4898130,6.480670e-02,6.499282e+02
age_groupElderly,1.000000e+00,2.412944,3.320822e-15,1.0000000,5.975125e-03,1.673605e+02
sexMale,6.439699e+00,1.762732,1.056588e+00,0.2906996,2.502528e-01,3.842656e+02
better_eye_catModerate,1.323156e+10,8497.994184,2.742514e-03,0.9978118,3.316770e-184,5.278454e+203
better_eye_catSevere,2.557659e+19,13651.665332,3.273462e-03,0.9973882,4.554678e-220,3.972707e+254
better_eye_catVery_poor,1.572235e+01,22782.444626,1.209301e-04,0.9999035,0.000000e+00,Inf
diagnosis_catConjunctival/surface disorders,8.306290e-20,12252.906304,-3.585655e-03,0.9971391,2.491565e-188,1.517345e+156


In [10]:
test_pred <- test_data %>%
  mutate(
    pred_prob = predict(model, newdata = test_data, type = "response"),
    pred_class = if_else(pred_prob >= 0.5, "1", "0"),
    pred_class = factor(pred_class, levels = c("0", "1"))
  )

conf_mat <- conf_mat(test_pred, truth = referral_needed, estimate = pred_class)
conf_mat

metrics(test_pred, truth = referral_needed, estimate = pred_class)


          Truth
Prediction  0  1
         0 35  2
         1  1 28

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9545455
kap,binary,0.9080780
